## Probability Distributions

author: Jacob Schreiber <br>
contact: jmschreiber91@gmail.com

Everything in pomegranate revolves around usage of probability distributions. Although these objects can be used by themselves, e.g., fit to data or given parameters and used to evaluate new examples, they are intended to be used as a part of a larger compositional model like a mixture or a hidden Markov model. Because everything in pomegranate is meant to be plug-and-play, this means that any probability distribution can be dropped into any other model. 

A key difference between distributions in torchegranate and those in pomegranate is that those in pomegranate are usually univariate, in that one object represents one dimension, whereas in torchegranate each distribution is multivariate. If you wanted to model several dimensions in pomegranate you would have to use an `IndependentComponentsDistribution` with many distributions dropped in, but in torchegranate you would use a single distribution object. We'll get more into that later. 

In [1]:
import numpy
import torch
from torchegranate.distributions import *

numpy.random.seed(0)
numpy.set_printoptions(suppress=True)

%load_ext watermark
%watermark -m -n -p numpy,torch,torchegranate

Populating the interactive namespace from numpy and matplotlib
numpy        : 1.23.4
scipy        : 1.9.3
torch        : 1.12.1
torchegranate: 0.0.1

Compiler    : GCC 11.2.0
OS          : Linux
Release     : 4.15.0-197-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit



### Initialization and Fitting

Let's first look at how to create a probability distribution. If you know what parameters you want to pass in, you can do that easily. These can be in the form of lists, tuples, numpy arrays, or torch tensors.

In [2]:
d1 = Normal([0.3, 0.7, 1.1], [1.1, 0.3, 1.8], covariance_type='diag')
d2 = Exponential([0.8, 1.4, 4.1])
d3 = Categorical([[0.3, 0.2, 0.5], [0.2, 0.1, 0.7]])

d11 = Normal((0.3, 0.7, 1.1), (1.1, 0.3, 1.8), covariance_type='diag')
d12 = Normal(numpy.array([0.3, 0.7, 1.1]), numpy.array([1.1, 0.3, 1.8]), covariance_type='diag')
d13 = Normal(torch.tensor([0.3, 0.7, 1.1]), torch.tensor([1.1, 0.3, 1.8]), covariance_type='diag')

If you don't have parameters you can learn them directly from data. Previously, this was done using the `Distribution.from_samples` method. However, because torchegranate aims to be more like sklearn, learning directly from data should just be done using `fit`. This will derive the parameters using MLE from the data.

In [3]:
numpy.random.seed(0)

X = numpy.random.randn(15, 3)

d4 = Normal().fit(X)
d4.means, d4.covs

(tensor([ 0.4604,  0.4067, -0.2158]),
 tensor([[ 1.3928, -0.0990,  0.0477],
         [-0.0990,  1.0676,  0.2146],
         [ 0.0477,  0.2146,  1.0698]]))

In [4]:
X2 = numpy.random.randint(3, size=(20, 4))

d5 = Categorical().fit(X2)
d5.probs

tensor([[0.4500, 0.2000, 0.3500],
        [0.4500, 0.2500, 0.3000],
        [0.2500, 0.3500, 0.4000],
        [0.2500, 0.2000, 0.5500]])

Similar to sklearn any hyperparameters used for training, such as regularization, will be passed into the initialization.  

### Probability and Log Probability

All distributions can calculate probabilities and log probabilities using those respective methods.

In [5]:
d4.log_probability(X)

tensor([-4.2034, -5.9343, -3.1988, -4.4541, -3.2705, -3.5384, -5.8283, -3.2755,
        -5.7044, -4.7418, -3.2053, -5.7131, -3.5851, -4.6144, -5.6844])

In [6]:
d1.log_probability(X)

tensor([ -3.6246,  -7.6793,  -4.2986,  -3.0519,  -3.2700,  -4.0210, -10.2286,
         -3.5409, -12.3042,  -3.7980,  -3.7762,  -6.9385,  -3.9249,  -9.5210,
         -7.6531])

In [7]:
d4.probability(X)

tensor([0.0149, 0.0026, 0.0408, 0.0116, 0.0380, 0.0291, 0.0029, 0.0378, 0.0033,
        0.0087, 0.0405, 0.0033, 0.0277, 0.0099, 0.0034])

In [8]:
d1.probability(X)

tensor([2.6660e-02, 4.6230e-04, 1.3587e-02, 4.7267e-02, 3.8006e-02, 1.7935e-02,
        3.6123e-05, 2.8986e-02, 4.5327e-06, 2.2415e-02, 2.2911e-02, 9.6973e-04,
        1.9744e-02, 7.3294e-05, 4.7457e-04])

 Similar to initialization, these can be lists, numpy arrays, or torch tensors.

In [9]:
d4.log_probability(torch.tensor(X))

tensor([-4.2034, -5.9343, -3.1988, -4.4541, -3.2705, -3.5384, -5.8283, -3.2755,
        -5.7044, -4.7418, -3.2053, -5.7131, -3.5851, -4.6144, -5.6844])

### Summarization

Although the primary way to learn parameters from data is to use the `fit` method, the underlying engine for this learning is a pair of operations: `summarize` and `from_summaries`. In `summarize`, the data is condensed into additive sufficient statistics that can be summed across batches. For instance:

In [10]:
d = Normal()
d.summarize(X[:5])

d._xw_sum

tensor([6.1267, 2.3821, 1.7964])

In [11]:
d.summarize(X[5:])

d._xw_sum

tensor([ 6.9055,  6.0998, -3.2373])

These values would be the same if we had summarized the entire data set.

In [12]:
d2 = Normal()
d2.summarize(X)

d2._xw_sum

tensor([ 6.9055,  6.0998, -3.2373])

From these values, usually stored as `_w_sum` and `_xw_sum`, one can perfectly recreate the values you would get if you fit to the entire data set. You can do this with the `from_summaries` method.

In [13]:
d.from_summaries()
d2.from_summaries()

d.means, d2.means

(tensor([ 0.4604,  0.4067, -0.2158]), tensor([ 0.4604,  0.4067, -0.2158]))

We will explore these ideas more in other tutorials, and specifically how this allows us to trivially implement batching schemes for out-of-core learning and for how to fit to large data sets using limited GPU memory.